In [4]:
!pip install langchain langchain-community openai duckduckgo-search

In [5]:
from langchain.agents import initialize_agent, Tool
from langchain.llms import OpenAI

In [3]:
openai_api_key = "**"

In [6]:
from langchain_community.tools.ddg_search import DuckDuckGoSearchRun

In [8]:
Search_tool = Tool(
    name = "DuckDuckGo Search",
    func=DuckDuckGoSearchRun().run,
    description="useful for when you need to answer questions about current events"
)

In [10]:
agent = initialize_agent(llm=OpenAI(temperature=0, api_key = openai_api_key),
                 tools=[Search_tool],
                 agent="zero-shot-react-description",
                 verbose=True)

In [15]:
response = agent.run("Who won Pak vs India Cricket Match Today(23/2/2025)? What is score of Virat Kohli?")



> Entering new AgentExecutor chain...
 I should search for the latest news and updates on the Pak vs India Cricket Match.
Action: DuckDuckGo Search
Action Input: 'Pak vs India Cricket Match Today(23/2/2025)'
Observation: Pakistan vs India live cricket score, Champions Trophy 2025, 5th Match, Group A at Dubai International Cricket Stadium. Get live score, ball by ball commentary, scorecard, match updates. Here are some key points surrounding today's India vs Pakistan match: ... News cricket India vs Pakistan ... Zimbabwe score after 2 overs is 14/2; Lifestyle News Live Today February 23, 2025 ... India chase 242 against Pakistan in their Group A match of the ICC Champions Trophy 2025 at the Dubai International Cricket Stadium. Saud Shakeel top scored for Pakistan with 62, while captain ... IND vs PAK CT 2025: India will take on Pakistan in their Group A encounter of the ICC Champions Trophy 2025 at the Dubai International Cricket Stadium on Sunday. The highly anticipated clash, set to

In [16]:
!pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=c833eee27e0e872a9f578ea3daa2c8ebd0580500e34984c08d0d3fb15a21f97c
  Stored in directory: /root/.cache/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia


In [17]:
from langchain_community.utilities.wikipedia import WikipediaAPIWrapper

In [19]:
wiki_tool = Tool(
    name = "Wikipedia",
    func=WikipediaAPIWrapper().run,
    description="useful for when you need to answer questions about current events"
)

In [24]:
llm=OpenAI(temperature=0, api_key = openai_api_key)

In [26]:
tools = [ wiki_tool]

In [27]:
new_agent = initialize_agent(
                 llm=llm,
                 tools=tools,
                 agent="zero-shot-react-description",
                 verbose=True)

In [29]:
new_agent.run("Who won Narispatnam assembly constituency in 2024 and what is the majority of winner?")



> Entering new AgentExecutor chain...
 I should check Wikipedia for information on the 2024 elections.
Action: Wikipedia
Action Input: "2024 elections Narispatnam assembly constituency"
Observation: Page: Second N. Chandrababu Naidu ministry
Summary: The Second N. Chandrababu Naidu ministry (or also known as 21st ministry of Andhra Pradesh) of the state of Andhra Pradesh was formed on 11 October 1999 headed by N. Chandrababu Naidu as the Chief Minister following the 1999 Andhra Pradesh Legislative Assembly election. The cabinet was sworn in a simple ceremony held at Raj Bhavan, Hyderabad. Initially the chief minister and eight other ministers of his council of ministers were administered the oath of office and secrecy by the then Governor C. Rangarajan. 
The cabinet was later expanded and reshuffled for four times citing several reasons on different occasions during the five year tenure to end on 14 May 2004.
Thought: I should check the results of the 2024 Andhra Pradesh Legislative 

'The Telugu Desam Party won the Narispatnam assembly constituency in the 2024 Andhra Pradesh Legislative Assembly election with a majority of 135 out of 144 contested seats.'

In [30]:
import sqlite3

# Connect to SQLite database (creates example.db if it doesn't exist)
connection = sqlite3.connect("example.db")
cursor = connection.cursor()

# Create the inventory table
cursor.execute("""
CREATE TABLE IF NOT EXISTS inventory (
    id INTEGER PRIMARY KEY,
    product_name TEXT NOT NULL,
    stock_quantity INTEGER NOT NULL
)
""")

# Insert sample data
cursor.executemany("""
INSERT INTO inventory (product_name, stock_quantity) VALUES (?, ?)
""", [
    ("Product A", 150),
    ("Product B", 200),
    ("Product C", 300),
    ("Product D", 100),
    ("Product E", 50)
])

# Commit and close connection
connection.commit()
connection.close()

print("Database created and populated with sample data.")

Database created and populated with sample data.


In [31]:
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit

In [33]:
connection = sqlite3.connect("example.db")
cursor = connection.cursor()


In [34]:
cursor.execute("SELECT * FROM inventory WHERE stock_quantity > 100")
rows = cursor.fetchall()

In [35]:
rows

[(1, 'Product A', 150), (2, 'Product B', 200), (3, 'Product C', 300)]

In [36]:
connection.commit()
connection.close()

In [37]:
# Step 1: Connect to the Database
db = SQLDatabase.from_uri("sqlite:///example.db")

In [40]:
# Step 2: Create the SQL Database Toolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

In [45]:
tools

[QuerySQLDatabaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7f7707c7ca50>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7f7707c7ca50>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7f7707c7ca50>),
 QuerySQLCheckerTool(description='Use this tool to double check

In [44]:
len(tools)

4

In [48]:
tools[0]

QuerySQLDatabaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7f7707c7ca50>)

In [49]:
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True
)

In [50]:
query1 = "How many units of Product A are in stock?"
response1 = agent.run(query1)
print("Response:", response1)



> Entering new AgentExecutor chain...
 I need to query the database to get the number of units of Product A in stock.
Action: sql_db_query
Action Input: SELECT units FROM products WHERE name = 'Product A'
Observation: Error: (sqlite3.OperationalError) no such table: products
[SQL: SELECT units FROM products WHERE name = 'Product A']
(Background on this error at: https://sqlalche.me/e/20/e3q8)
Thought: I need to use sql_db_list_tables to see what tables are in the database.
Action: sql_db_list_tables
Action Input: ""
Observation: inventory
Thought: I need to use sql_db_schema to see the schema and sample rows for the inventory table.
Action: sql_db_schema
Action Input: inventory
Observation: 
CREATE TABLE inventory (
	id INTEGER, 
	product_name TEXT NOT NULL, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (id)
)

/*
3 rows from inventory table:
id	product_name	stock_quantity
1	Product A	150
2	Product B	200
3	Product C	300
*/
Thought: I need to use sql_db_query_checker to double chec